In [1]:
import xarray as xr
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

%load_ext autoreload
%autoreload 2

### Paths input data

In [2]:
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, RESULTS_DIR

# PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
# PATH_DT = OUTPUT_DATA_DIR / '/dT_data_rcmip_models.nc'
PATH_DT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


#### Uncertainty data from Chris

In [3]:
PATH_DT_UNCERTAINTY = OUTPUT_DATA_DIR / 'dT_uncertainty_data_FaIR_chris_ed02-3.nc'

## Set values:

In [4]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [5]:
ref_year = 2019

In [6]:
FIGURE_DIR = RESULTS_DIR / 'figures_recommendation/'

TABLE_DIR = RESULTS_DIR / 'tables_recommendation/'

In [7]:
from pathlib import Path
Path(FIGURE_DIR).mkdir(parents=True, exist_ok=True)
Path(TABLE_DIR).mkdir(parents=True, exist_ok=True)

In [8]:
percentile = 'percentile'
climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
name_deltaT = 'Delta T'


### Define variables to look at:

In [9]:
# variables to plot:
variables_erf_comp = [
    'ch4',
    'aerosol-total-with_bc-snow',
    #'aerosol-radiation_interactions',
    #'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    # 'F-Gases|HFC',
    'bc_on_snow',
    'total_anthropogenic',
    #'total',
]
variables_in_sum = [
    'aerosol-total-with_bc-snow',
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    #'aerosol-total',
    'o3',
    'HFCs',
    #'bc_on_snow'
]

# total ERFs for anthropogenic and total:
#variables_erf_tot = ['total_anthropogenic',
#                     'total']

scenarios_fl_370 = ['ssp370', 'ssp370-lowNTCF-aerchemmip', 'ssp370-lowNTCF-gidden'  # Due to mistake here
                    ]

### Scenarios:

In [10]:
scenarios_fl = ['ssp119',
                'ssp126',
                'ssp245',
                'ssp370',
                'ssp370-lowNTCF-aerchemmip',
                #'ssp370-lowNTCF-gidden',
                'ssp370-lowNTCF-gidden',
                'ssp585'
               ]

In [11]:
median = 'median'
perc5 = '5th percentile'
perc95 = '95th percentile'
recommendation = 'recommendation'

## Open dataset:

### Integrate:
The code below opens the file generated in [2_compute_delta_T.ipynb](2_compute_delta_T.ipynb) by integrating

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

where IRF is the impulse response function and ERF is the effective radiative forcing from RCMIP. 

In [12]:
ds_DT = xr.open_dataset(PATH_DT)
ds_uncertainty = xr.open_dataset(PATH_DT_UNCERTAINTY)

In [13]:
for var in variables_erf_comp:
    da5 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p05-p50']    
    da95 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p95-p50']
    da5.plot(label=var)
    da95.plot(label=var)

In [14]:
ds_uncertainty.variable

<xarray.DataArray 'variable' (variable: 8)>
array(['HFCs', 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)', 'aerosol-total',
       'bc_on_snow', 'ch4', 'o3', 'total_anthropogenic',
       'aerosol-total-with_bc-snow'], dtype=object)
Coordinates:
  * variable  (variable) object 'HFCs' ... 'aerosol-total-with_bc-snow'

In [15]:
v_sum = 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'
ds_uncertainty['p05-p50'].sel(year=2040, scenario='ssp119', variable=v_sum)#.variables#.plot()#, variable='%%SVG')%%SVG

<xarray.DataArray 'p05-p50' (base_period: 1)>
array([-0.126748])
Coordinates:
    year         int64 2040
  * base_period  (base_period) int64 2020
    scenario     <U6 'ssp119'
    variable     <U41 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'

In [16]:

ds_uncertainty.variable

<xarray.DataArray 'variable' (variable: 8)>
array(['HFCs', 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)', 'aerosol-total',
       'bc_on_snow', 'ch4', 'o3', 'total_anthropogenic',
       'aerosol-total-with_bc-snow'], dtype=object)
Coordinates:
  * variable  (variable) object 'HFCs' ... 'aerosol-total-with_bc-snow'

In [17]:
ds_DT

<xarray.Dataset>
Dimensions:     (percentile: 1, scenario: 11, variable: 19, year: 351)
Coordinates:
  * year        (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * variable    (variable) object 'co2' 'ch4' ... 'HFCs'
  * scenario    (scenario) object 'ssp534-over' ... 'ssp370-lowNTCF-gidden'
  * percentile  (percentile) object 'recommendation'
Data variables:
    ERF         (scenario, variable, year) float64 ...
    time        (year) datetime64[ns] ...
    delta_t     (year) float64 ...
    Delta T     (percentile, scenario, variable, year) float64 ...

## Merge uncertainty and original: 


In [18]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

<IPython.core.display.Javascript object>

hallo


### Add sum

In [19]:
_str = ''
_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_in_sum]
for var in _vl: 
    _str += f'{var}, '

# ax.set_title('Temperature change, sum SLCF  (%s)' % _str[:-2])


vn_sum = 'Sum SLCF (%s)' % _str[:-2]
print(vn_sum)
#_st = vn_sum.replace('(','').replace(')','').replace(' ','_').replace(',','')+'.csv'

_da_sum  = ds_DT[name_deltaT].sel(variable=variables_in_sum).sum(variable)
#_da = ds_DT[name_deltaT].sel(variable=variables_erf_comp).sum(variable).sel(year=slice(int(s_y2), int(e_y2))) - ds_DT_sy
_da_sum#.assin_coord()
#_ds_check = ds_DT.copy()
ds_DT
#xr.concat([_ds_check[name_deltaT],_da_sum], dim=variable)

dd1=_da_sum.expand_dims(
    {'variable':
    [vn_sum]})
#dd1=dd1.to_dataset()

ds_DT = xr.merge([ds_DT,dd1])

Sum SLCF (Aerosols, Methane, Ozone, HFCs)


In [20]:
percentiles_to_keep = ['p05-p50','p16-p50','p84-p50','p95-p50']

In [21]:
_da = ds_uncertainty[percentiles_to_keep].to_array('percentile')
_ds = _da.rename(name_deltaT).to_dataset()

In [22]:
ds_DT= xr.concat([ds_DT[name_deltaT],_ds[name_deltaT]], dim='percentile').to_dataset()

In [23]:
ds_DT

<xarray.Dataset>
Dimensions:      (base_period: 1, percentile: 5, scenario: 11, variable: 20, year: 351)
Coordinates:
  * variable     (variable) object 'HFCs' ... 'volcanic'
  * scenario     (scenario) object 'ssp119' 'ssp126' ... 'ssp534-over' 'ssp585'
  * year         (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * base_period  (base_period) int64 2020
  * percentile   (percentile) object 'recommendation' 'p05-p50' ... 'p95-p50'
Data variables:
    Delta T      (percentile, scenario, variable, year, base_period) float64 0.0 ... nan

In [24]:
ds_DT.sel(year=2040, scenario='ssp119', percentile='recommendation', variable=v_sum)#.variables#.plot()#, variable='%%SVG')%%SVG

<xarray.Dataset>
Dimensions:      (base_period: 1)
Coordinates:
    variable     <U41 'Sum SLCF (Aerosols, Methane, Ozone, HFCs)'
    scenario     <U6 'ssp119'
    year         int64 2040
  * base_period  (base_period) int64 2020
    percentile   <U14 'recommendation'
Data variables:
    Delta T      (base_period) float64 0.1433

## Make csv table of total values:

In [25]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

In [26]:
reference_year = ref_year
start_y_tabel = 2015
end_y_tabel = last_y

In [27]:
def get_fn(var_name, s_y, e_y, ref_y, perc):
    _st = var_name.replace('(','').replace(')','').replace(' ','_').replace(',','')#+'.csv'
    fn = f'{perc}_{_st}_{s_y}-{e_y}_refyear{ref_y}.csv'
    return fn

def make_sum_slcfs_tabel(sum_vn = vn_sum, percentile=recommendation):
    #_str = ''
    #_vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_erf_comp]
    #for var in _vl:
    #    _str += f'{var}, '
    #vn_sum = 'Sum SLCF (%s)' % _str[:-2]

    fn = get_fn(vn_sum, start_y_tabel, end_y_tabel, ref_year, percentile)



    # ref year value:
    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=sum_vn,
        year=reference_year
    ).squeeze()

    # all values from s_y to e_y
    _da = ds_DT[name_deltaT].sel(
        variable=sum_vn,
        year=slice(start_y_tabel, end_y_tabel)
    ) - ds_DT_sy

    # Choose recommendation::
    _pl_da = _da.sel(percentile=percentile).squeeze()
    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    fn = TABLE_DIR / fn
    print(percentile)
    display(df.loc[2040])
    df.to_csv(fn)
    
    
#for prc in [recommendation, 'p05-p50','p95-p50']:
#    make_sum_slcfs_tabel(percentile=prc)

In [28]:
import pandas as pd

## TABLE EACH VAR

In [29]:

def make_slcfs_tabel(var, percentile=recommendation):
    _str = ''
    _st = get_fn(var, start_y_tabel, end_y_tabel, ref_year, percentile)


    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=var, 
        year=reference_year,
    ).squeeze()
    
    _da = ds_DT[name_deltaT].sel(
        variable=var,
        year=slice(start_y_tabel, end_y_tabel)) - ds_DT_sy
    
    # Take recommendation::
    _pl_da = _da.sel(percentile=recommendation).squeeze()

    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    display(df)
    fn = TABLE_DIR/_st
    df.to_csv(fn)

for var in variables_erf_comp+[vn_sum]:
    for prc in [recommendation, 'p05-p50','p95-p50']:
        make_slcfs_tabel(var, percentile=prc)

scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,recommendation
2016,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,recommendation
2017,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,recommendation
2018,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,recommendation
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157570,-0.149586,0.004332,-0.001586,0.273598,0.273598,-0.122313,0.132585,0.186522,-0.161157,0.159666,recommendation
2097,-0.158986,-0.151116,0.003826,-0.001399,0.276558,0.276558,-0.121914,0.132488,0.186616,-0.162629,0.158069,recommendation
2098,-0.160369,-0.152600,0.003354,-0.001206,0.279497,0.279497,-0.121485,0.132353,0.186696,-0.164022,0.156481,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,p05-p50
2016,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,p05-p50
2017,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,p05-p50
2018,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,p05-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157570,-0.149586,0.004332,-0.001586,0.273598,0.273598,-0.122313,0.132585,0.186522,-0.161157,0.159666,p05-p50
2097,-0.158986,-0.151116,0.003826,-0.001399,0.276558,0.276558,-0.121914,0.132488,0.186616,-0.162629,0.158069,p05-p50
2098,-0.160369,-0.152600,0.003354,-0.001206,0.279497,0.279497,-0.121485,0.132353,0.186696,-0.164022,0.156481,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,-0.007652,p95-p50
2016,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,-0.005760,p95-p50
2017,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,-0.003890,p95-p50
2018,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,-0.001966,p95-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157570,-0.149586,0.004332,-0.001586,0.273598,0.273598,-0.122313,0.132585,0.186522,-0.161157,0.159666,p95-p50
2097,-0.158986,-0.151116,0.003826,-0.001399,0.276558,0.276558,-0.121914,0.132488,0.186616,-0.162629,0.158069,p95-p50
2098,-0.160369,-0.152600,0.003354,-0.001206,0.279497,0.279497,-0.121485,0.132353,0.186696,-0.164022,0.156481,p95-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,recommendation
2016,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,recommendation
2017,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,recommendation
2018,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,recommendation
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.434047,0.442553,0.287119,0.295194,-0.042055,0.280523,0.280523,0.251682,0.170740,0.383187,0.235081,recommendation
2097,0.436851,0.445482,0.289478,0.297135,-0.040443,0.282350,0.282350,0.255310,0.173814,0.383637,0.239896,recommendation
2098,0.439666,0.448390,0.291762,0.299069,-0.038798,0.284184,0.284184,0.258967,0.176847,0.384078,0.244763,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,-0.049092,p05-p50
2016,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,-0.033405,p05-p50
2017,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,-0.018735,p05-p50
2018,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,-0.008387,p05-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.434047,0.442553,0.287119,0.295194,-0.042055,0.280523,0.280523,0.251682,0.170740,0.383187,0.235081,p05-p50
2097,0.436851,0.445482,0.289478,0.297135,-0.040443,0.282350,0.282350,0.255310,0.173814,0.383637,0.239896,p05-p50
2098,0.439666,0.448390,0.291762,0.299069,-0.038798,0.284184,0.284184,0.258967,0.176847,0.384078,0.244763,p05-p50


...

scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,p05-p50
2016,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,p05-p50
2017,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,p05-p50
2018,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,p05-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.178680,0.606133,1.783780,0.916636,3.042914,3.273967,2.557981,1.149115,2.265593,0.997438,4.022242,p05-p50
2097,0.172654,0.601335,1.793323,0.917357,3.089195,3.319977,2.598586,1.147042,2.282381,0.984752,4.075115,p05-p50
2098,0.166555,0.596669,1.802563,0.918045,3.135628,3.366110,2.639395,1.144516,2.298805,0.972196,4.127534,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,-0.155669,p95-p50
2016,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,-0.113028,p95-p50
2017,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,-0.072562,p95-p50
2018,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,-0.035559,p95-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.178680,0.606133,1.783780,0.916636,3.042914,3.273967,2.557981,1.149115,2.265593,0.997438,4.022242,p95-p50
2097,0.172654,0.601335,1.793323,0.917357,3.089195,3.319977,2.598586,1.147042,2.282381,0.984752,4.075115,p95-p50
2098,0.166555,0.596669,1.802563,0.918045,3.135628,3.366110,2.639395,1.144516,2.298805,0.972196,4.127534,p95-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,recommendation
2016,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,recommendation
2017,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,recommendation
2018,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,recommendation
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,recommendation
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.121184,0.142532,0.305647,0.217473,0.560201,0.801208,0.140874,0.430939,0.502833,0.107733,0.681819,recommendation
2097,0.122170,0.143131,0.307001,0.219706,0.568515,0.809549,0.144191,0.433513,0.504457,0.105918,0.683663,recommendation
2098,0.123234,0.143793,0.308340,0.221953,0.576844,0.817878,0.147572,0.436083,0.506058,0.104248,0.685475,recommendation


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,p05-p50
2016,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,p05-p50
2017,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,p05-p50
2018,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,p05-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.121184,0.142532,0.305647,0.217473,0.560201,0.801208,0.140874,0.430939,0.502833,0.107733,0.681819,p05-p50
2097,0.122170,0.143131,0.307001,0.219706,0.568515,0.809549,0.144191,0.433513,0.504457,0.105918,0.683663,p05-p50
2098,0.123234,0.143793,0.308340,0.221953,0.576844,0.817878,0.147572,0.436083,0.506058,0.104248,0.685475,p05-p50


scenario,ssp119,ssp126,ssp245,ssp334,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,,
2015,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,-0.066640,p95-p50
2016,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,-0.046757,p95-p50
2017,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,-0.028583,p95-p50
2018,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,-0.013701,p95-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.121184,0.142532,0.305647,0.217473,0.560201,0.801208,0.140874,0.430939,0.502833,0.107733,0.681819,p95-p50
2097,0.122170,0.143131,0.307001,0.219706,0.568515,0.809549,0.144191,0.433513,0.504457,0.105918,0.683663,p95-p50
2098,0.123234,0.143793,0.308340,0.221953,0.576844,0.817878,0.147572,0.436083,0.506058,0.104248,0.685475,p95-p50


In [ ]:
print('done')